In [44]:
# !pip install Praw
# !pip install --upgrade praw

Requirement already up-to-date: praw in c:\users\tiernon\anaconda3\lib\site-packages (6.5.1)


In [55]:
import praw
import pandas as pd
import datetime as dt

f = open("../secret_python.txt", "r")
client_id = f.readline().rstrip()
secret_key = f.readline().rstrip()
f.close()

In [56]:
reddit = praw.Reddit(client_id=client_id, \
                     client_secret=secret_key, \
                     user_agent='Scrape')

reddit.read_only

True

In [57]:
me_irl = reddit.subreddit('me_irl')

In [58]:
top_me_irl = me_irl.top(limit=20)

In [59]:
for submission in top_me_irl:
    print(submission.title, submission.id, submission.score)

Me📢irl cs507j
